In [1]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.1 MB/s 
     |████████████████████████████████| 209 kB 51.1 MB/s 
     |████████████████████████████████| 81 kB 9.6 MB/s 
     |████████████████████████████████| 78 kB 6.8 MB/s 
     |████████████████████████████████| 112 kB 61.3 MB/s 
     |████████████████████████████████| 146 kB 44.3 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=34dfd041df5cb1d7de7156683fe82c6aaf3cdcae4b13d73545d871b4ca8c8ddb
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [2]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMClassifier
from lightgbm.sklearn import LGBMRegressor

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

import optuna

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
base_path='/content/drive/MyDrive/minipjt34/'

In [5]:
# 데이터를 불러옵니다.
train =pd.read_csv(base_path + 'train_V2.csv')
test = pd.read_csv(base_path + 'test_V2.csv')
submission = pd.read_csv(base_path + 'sample_submission_V2.csv')

In [6]:
temp1=train.copy()

In [9]:
#데이터크기축소
temp1 = temp1.sample(frac=0.05, random_state=42, replace=False)

In [10]:
len(temp1)

22235

In [11]:
# 이벤트 제거
temp2 = temp2.drop(temp2[temp2.matchType.str.contains("flare")].index, axis=0)
temp2 = temp2.drop(temp2[temp2.matchType.str.contains("crash")].index, axis=0)
temp2

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
4289863,ab772932dc284b,6f0931849c42fc,7558d7990b8795,0,0,161.3,3,2,0,7,...,1,0.0,0,0.0,0,0,565.3,2,1660,0.3929
1660760,eae2ba6cc7de4c,a8ddca0c7e8f74,0cf8e1dc70559e,0,2,100.0,0,0,1,24,...,0,450.7,0,0.0,0,0,1607.0,6,1593,0.7083
363725,aa5867db86503b,76d57fc80f70f2,fc69844a437eae,3,2,289.9,2,0,9,14,...,1,3950.0,0,0.0,0,0,2131.0,5,0,0.8148
1073463,9ec6c0ee672370,78935f8da2a64d,8a6b99c6d832f1,1,0,179.5,1,0,0,69,...,1,0.0,0,0.0,0,0,837.6,4,0,0.3333
414963,59f1f08405cf55,45bd8c2766b800,d39627df3d2674,0,0,114.8,0,0,0,71,...,0,0.0,0,0.0,0,0,187.4,2,1526,0.3214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2381585,5b4159c4065c3e,8f1920af94375d,44366e01c3cece,0,2,0.0,0,0,6,56,...,0,972.3,0,0.0,0,0,1145.0,2,0,0.6000
3010658,f402fa09f3f959,825c60ecc6bf1b,ef048c066bf2f5,0,0,416.0,3,0,0,8,...,0,0.0,0,0.0,0,0,2717.0,6,1483,0.9200
298060,51628215e297ac,93a24c14ae4457,61a07c655772b3,0,0,111.4,1,0,4,45,...,0,5139.0,0,0.0,0,0,3211.0,5,1565,0.7407
3190576,e60f2d43169e35,615e603e029639,7ad802e3c51d18,0,1,0.0,0,0,5,54,...,0,1940.0,0,0.0,1,1,2136.0,5,0,0.4800


In [12]:
temp2 = temp1.iloc[:,3:]
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(temp2['matchType'])
encoder.fit(test["matchType"])
temp2['matchType'] = encoder.transform(temp2['matchType'])
test['matchType'] = encoder.transform(test['matchType'])

In [13]:
trainX = temp2.drop(['winPlacePerc','killPlace'], axis=1)
testX = test.drop(columns='killPlace').iloc[:,3:]
y = temp2.winPlacePerc
print(trainX.shape, testX.shape, y.shape)

(22235, 24) (1934174, 24) (22235,)


In [14]:
#vif 알아보기

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    trainX.values, i) for i in range(trainX.shape[1])]
vif["features"] = trainX.columns
vif

,VIF Factor,features
0,1.478254,assists
1,3.307610,boosts
2,9.830654,damageDealt
3,3.994108,DBNOs
4,2.140526,headshotKills
5,1.868521,heals
6,54.947071,killPoints
7,11.358821,kills
8,4.803585,killStreaks
9,2.072788,longestKill


In [15]:
from statsmodels.regression.linear_model import OLS

In [16]:
import statsmodels.api as sm; 

In [17]:
feature_1 = sm.add_constant(trainX)
model_PU = sm.OLS(y, feature_1)
result_PU = model_PU.fit()
print(result_PU.summary())

                            OLS Regression Results                            
Dep. Variable:           winPlacePerc   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.752
Method:                 Least Squares   F-statistic:                     2816.
Date:                Mon, 13 Jun 2022   Prob (F-statistic):               0.00
Time:                        13:52:24   Log-Likelihood:                 10170.
No. Observations:               22235   AIC:                        -2.029e+04
Df Residuals:                   22210   BIC:                        -2.009e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.2118      0.021     

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [27]:
trainX.columns

Index(['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
       'killPoints', 'kills', 'killStreaks', 'longestKill', 'matchDuration',
       'matchType', 'maxPlace', 'numGroups', 'rankPoints', 'revives',
       'rideDistance', 'roadKills', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints'],
      dtype='object')

In [ ]:
# vif 가 높은 컬럼 maxPlace 를 삭제함

In [46]:
cols = ['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
       'killPoints', 'kills', 'killStreaks', 'longestKill', 'matchDuration',
       'matchType', 'numGroups', 'rankPoints', 'revives',
       'rideDistance', 'roadKills', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints']
feature_1 = sm.add_constant(trainX[cols])
model_PU2 = sm.OLS(y, feature_1[cols])
result_PU2 = model_PU.fit()
print(result_PU2.summary2())



/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                   Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.762       
Dependent Variable: winPlacePerc     AIC:                -585790.9598
Date:               2022-06-13 04:11 BIC:                -585519.4111
No. Observations:   605937           Log-Likelihood:     2.9292e+05  
Df Model:           23               F-statistic:        8.452e+04   
Df Residuals:       605913           Prob (F-statistic): 0.00        
R-squared:          0.762            Scale:              0.022267    
---------------------------------------------------------------------
                     Coef.  Std.Err.     t     P>|t|   [0.025  0.975]
---------------------------------------------------------------------
const                0.2405   0.0040   60.4501 0.0000  0.2327  0.2483
assists              0.0155   0.0004   40.8696 0.0000  0.0147  0.0162
boosts               0.0138   0.0002   79.3586 0.0000  0.0134  0.0141
damageDealt          0.0001   0.0000   

In [ ]:
trainX.drop(columns=["maxPlace"], inplace=True)

In [48]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    trainX.values, i) for i in range(trainX.shape[1])]
vif["features"] = trainX.columns
vif

,VIF Factor,features
0,1.468626,assists
1,3.375589,boosts
2,9.479082,damageDealt
3,4.072519,DBNOs
4,2.085517,headshotKills
5,1.875379,heals
6,54.722543,killPoints
7,11.296902,kills
8,4.989915,killStreaks
9,2.109336,longestKill


In [40]:
vif.sort_values(by="VIF Factor", ascending=False)

,VIF Factor,features
22,77.476663,winPoints
6,54.722543,killPoints
10,42.356455,matchDuration
13,31.697609,rankPoints
7,11.296902,kills
2,9.479082,damageDealt
12,5.972234,numGroups
21,5.466549,weaponsAcquired
11,5.456742,matchType
8,4.989915,killStreaks


In [56]:
#p score 에서 무의미한 데이터 제거 roadKills

trainX.drop("kills", axis=1, inplace=True)


In [57]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    trainX.values, i) for i in range(trainX.shape[1])]
vif["features"] = trainX.columns
vif

,VIF Factor,features
0,1.452314,assists
1,3.286909,boosts
2,6.567304,damageDealt
3,4.005235,DBNOs
4,1.891919,headshotKills
5,1.870847,heals
6,5.932417,killPoints
7,3.697566,killStreaks
8,2.033109,longestKill
9,5.354810,matchType


In [58]:
vif.sort_values(by="VIF Factor", ascending=False)

,VIF Factor,features
11,8.589388,rankPoints
2,6.567304,damageDealt
6,5.932417,killPoints
10,5.433136,numGroups
18,5.355230,weaponsAcquired
9,5.354810,matchType
17,4.401002,walkDistance
3,4.005235,DBNOs
7,3.697566,killStreaks
1,3.286909,boosts


In [62]:
trainX.columns

Index(['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
       'killPoints', 'killStreaks', 'longestKill', 'matchType', 'numGroups',
       'rankPoints', 'revives', 'rideDistance', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired'],
      dtype='object')

In [ ]:
Index(['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
       'killPlace', 'killPoints', 'killStreaks', 'longestKill', 'matchType',
       'numGroups', 'revives', 'rideDistance', 'roadKills', 'swimDistance',
       'teamKills', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired'],
      dtype='object')

In [63]:
trainX

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPoints,killStreaks,longestKill,matchType,numGroups,rankPoints,revives,rideDistance,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired
4289863,0,0,161.3,3,2,0,1607,3,41.54,15,29,-1,1,0.0,0.0,0,0,565.3,2
1660760,0,2,100.0,0,0,1,1700,1,64.25,13,97,-1,0,450.7,0.0,0,0,1607.0,6
363725,3,2,289.9,2,0,9,0,1,22.46,15,26,1456,1,3950.0,0.0,0,0,2131.0,5
1073463,1,0,179.5,1,0,0,0,0,0.00,15,25,1456,1,0.0,0.0,0,0,837.6,4
414963,0,0,114.8,0,0,0,1326,0,0.00,15,29,-1,0,0.0,0.0,0,0,187.4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2381585,0,2,0.0,0,0,6,0,0,0.00,15,26,1484,0,972.3,0.0,0,0,1145.0,2
3010658,0,0,416.0,3,0,0,1378,2,70.18,14,25,-1,0,0.0,0.0,0,0,2717.0,6
298060,0,0,111.4,1,0,4,1336,0,0.00,15,26,-1,0,5139.0,0.0,0,0,3211.0,5
3190576,0,1,0.0,0,0,5,0,0,0.00,14,26,1486,0,1940.0,0.0,1,1,2136.0,5


In [66]:
testX=testX[['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
       'killPoints', 'killStreaks', 'longestKill', 'matchType', 'numGroups',
       'rankPoints', 'revives', 'rideDistance', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired']]

In [67]:
print(trainX.shape, testX.shape, y.shape)

(605937, 19) (1934174, 19) (605937,)
